In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [13]:
# Get the same results each time
np.random.seed(0)


# Load the training data
data = pd.read_csv("data/data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

## Задание 1: Разделение данных на train и test

In [14]:
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.3, random_state=42)

## Задание 2: Преобразование текста в числовой формат

In [15]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

## Задание 3: Обучение модели логистической регрессии

In [16]:
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train_vec, y_train)

y_pred = classifier.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9273


## Задание 4: Функция для предсказания токсичности

In [17]:
def predict(comment):
    comment_vec = vectorizer.transform([comment])
    res = classifier.predict_proba(comment_vec)[0][1]
    return res

## Задание 5: Тестирование модели

In [18]:
print("'Apples are stupid':", predict("Apples are stupid"))
print("'I love apples':", predict("I love apples"))

'Apples are stupid': 0.9991497306309441
'I love apples': 0.05671043399691718


## Задание 6: Вывод самых токсичных слов

In [19]:
feature_names = vectorizer.get_feature_names_out()
coefs = classifier.coef_[0]
top_toxic_indices = coefs.argsort()[-10:][::-1]

print("Top 10 toxic words:")
for idx in top_toxic_indices:
    print(f"{feature_names[idx]}: {coefs[idx]:.4f}")

Top 10 toxic words:
stupid: 9.2929
idiot: 8.8017
idiots: 8.5161
stupidity: 7.5930
idiotic: 6.8486
crap: 6.5977
dumb: 6.4676
hypocrite: 6.4396
pathetic: 6.4346
moron: 6.4076


## Задание 7: Анализ токсичных слов

В списке есть многозначные слова, которые могут быть токсичными в одном контексте и совершенно нетоксичными в другом (crap, dumb, pathetic).
Из-за того, что они имеют высокий "рейтинг" токсичности, наверняка любая фраза с их участием в любом контексте будет помечаться токсичной.
Для иллюстрации взял слово dumb в значении `немой` и первую фразу с ним из Яндекс-переводчика (цитата из "Три товарища" Ремарка)

In [20]:
print(predict('It was as if some poor, dumb human being, pent up in the crooked body, were trying to free and save itself'))

0.9964174458999558


## Задание 8: Проверка на предвзятость

In [21]:
comments_to_test = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend"
]

for comment in comments_to_test:
    print(f"'{comment}': {predict(comment):.4f}")

'I have a christian friend': 0.1881
'I have a muslim friend': 0.5027
'I have a white friend': 0.3928
'I have a black friend': 0.5798


Видим, что алгоритм явно предвзято относится к исламу и черным.

## Задание 9: Анализ типа предвзятости

Это пример исторической предвзятости (historical bias), когда модель отражает существующие в обществе стереотипы. В данном случае модель может показывать исламофобные тенденции, так как обучалась на данных, содержащих такие предубеждения.

## Задание 10: Идеи для улучшения

1. Балансировка данных - убедиться, что все группы представлены равномерно
2. Удаление или понижение веса токсичных слов, связанных с конкретными группами